In [1]:
import sys
sys.path.append('../')
import importlib
import models.transunet as vit
import models.decoder_layers as decoder_layers
import models.decoder_layers as encoder_layers
import models.utils as ut
import experiments.config as conf
# import data_processing.dataset_synapse as dp 
import data_processing.data_parser as dp
import data_processing.dataset_synapse as ds
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa
import models.resnet_v2 as res

tfk = tf.keras

In [114]:
conv = tfk.layers.Conv2D(64,3,use_bias=False)
gn = tfa.layers.GroupNormalization(64)
_ = gn(conv(dummy))


In [94]:
y = tf.keras.layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)
vprint(y)

tf.Tensor(
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 0  0  0  0]]

  [[ 8  9 10 11]
   [12 13 14 15]
   [ 0  0  0  0]]

  [[ 0  0  0  0]
   [ 0  0  0  0]
   [ 0  0  0  0]]]], shape=(1, 3, 3, 4), dtype=int32)


In [148]:
weights = np.load("../data/R50+ViT-B_16.npz")
# weights.files

In [150]:
importlib.reload(res)
model = res.ResNetV2([3,4,9])
model(dummy)
model.load_weights(weights)

In [134]:
test = np.zeros((1,1,1,64))
np.squeeze(test, axis=(0,1,2)).shape

(64,)

In [85]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
t.shape

TensorShape([2, 3])

In [2]:
X, y = ds.load_data("../data/train_npz/", 10, output_size=224)

In [8]:
print(X.shape)
print(y.shape)

(10, 384, 384, 3)
(10, 384, 384, 9)


In [2]:
resnet = tfk.applications.ResNet50V2(include_top=False, input_shape=(224,224,3))

In [37]:
w = resnet.get_layer('conv1_conv').weights[0]
m, v = tf.nn.moments(w, axes=[0,1,2], keepdims=True) 
w = (w-m) / tf.sqrt(v+1e-5)
tf.math.is_nan(w)
w.shape


TensorShape([7, 7, 3, 64])

In [60]:
class StdConv2D(tfkl.Conv2D):
    
    def call(self, x):
        w = self.weights[0]
        m, v = tf.nn.moments(w, axes=[0,1,2], keepdims=True) 
        w = (w-m) / tf.sqrt(v+1e-5)
        # self.weights = w 
        return tf.nn.conv2d(x, w, self.strides, self.padding.upper(), "NHWC" if self.data_format == "channels_last" else "NCHW", self.dilation_rate, self.name)

In [67]:
conv = StdConv2D(filters=64, kernel_size=7, strides=2, padding="same")
dummy = np.random.rand(1,224,224,3)
# conv(dummy)

In [3]:
resnet.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [3]:
importlib.reload(conf)
importlib.reload(decoder_layers)
importlib.reload(encoder_layers)
importlib.reload(vit)
importlib.reload(ut)
importlib.reload(res)
config = conf.get_r50_b16()
trans = vit.TransUnet(config)
trans.compile()

(None, 224, 224, 3) Tensor("resnet_v2/Shape:0", shape=(4,), dtype=int32)
461217792/461217452 [==============================] - 57s 0us/step


In [4]:
trans.model.fit(x=X, y=y, epochs=10, batch_size=24, verbose=1)

Epoch 1/10
(None, 224, 224, 3) Tensor("R50-B16_CUP/resnet_v2/Shape:0", shape=(4,), dtype=int32)
(None, 224, 224, 3) Tensor("R50-B16_CUP/resnet_v2/Shape:0", shape=(4,), dtype=int32)


In [11]:
importlib.reload(conf)
importlib.reload(decoder_layers)
importlib.reload(encoder_layers)
importlib.reload(vit)
importlib.reload(ut)
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()